In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import path
import os

# Classifiers
from sklearn.naive_bayes import GaussianNB # Gaussiana de Naive-Bayes
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron
from sklearn.neighbors import KNeighborsClassifier # K-Nearest Neighbors
from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.ensemble import AdaBoostClassifier # AdaBoost
from sklearn.svm import SVC # Support Vector Machine (Linear, Polynomial, RBF)

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix

## Classification of datasets

In [2]:
EXPERIMENT_NAME = "AcY_Classification"

# Set datasets to classification: ['Raw', 'Normalized', 'Fourier', 'HOS', 'SCM']
datasets_name_list = ['SCM']

# Set classifiers to classification: ['Naive_Bayes','MLP','Nearest_Neighbors','Random_Forest','SVM_Linear','SVM_Polynomial','SVM_RBF']
classifiers_name_list = ['Naive_Bayes','MLP','Nearest_Neighbors','Random_Forest','SVM_Linear','SVM_Polynomial','SVM_RBF']

# Set metrics to evaluate classifier performance: ['accuracy','balanced_accuracy','precision','sensitivity','specificity','f1_score','fit_time','predict_time']
metrics_name_list = ['accuracy','precision','sensitivity','specificity','f1_score','fit_time','predict_time']

# Set classes to classification: 3225 = CLASS 0, 3225_REVERSO = CLASS 1, 3225_VAZIO = CLASS 2, 3230 = CLASS 3, 3230_VAZIO = CLASS 4, 3235 = CLASS 5
classes = [0, 2, 3, 4, 5]

group_classes = True # Grouping classes
balance_classes = True # Balance classes
classification_type = 'KFold_Test' # 'KFold_Test_Val', 'KFold_Test', 'HoldOut'

# Define the grouping, if it is True
if group_classes:
    changeClass_dict = {0:0, 2:1, 3:2, 4:3, 5:4} # format {old:new, old:new,...}

In [3]:
# Making dirs to save results
outpath = path.join("..","results","classification",EXPERIMENT_NAME)
os.makedirs(outpath)

train_path = path.join(outpath, 'Train')
os.makedirs(train_path)

if classification_type == 'KFold_Test_Val':
    validation_path = path.join(outpath, 'Validation')
    os.makedirs(validation_path)

for ds_name in datasets_name_list:
    os.makedirs(path.join(train_path, ds_name))
    if classification_type == 'KFold_Test_Val':
        os.makedirs(path.join(validation_path, ds_name))
    
def join_classes(df, dict_groups):
    df['Class'].replace(dict_groups, inplace=True)
    return df

In [4]:
if balance_classes:
    list_drops = [0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29, 30, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 97, 98, 100, 101, 102, 103, 104, 106, 107, 108, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 180, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 250, 251, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264, 265, 267, 268, 269, 270, 271, 272, 274, 275, 276, 278, 279, 280, 281, 282, 283, 285, 286, 288, 289, 290, 291, 292, 293, 294, 296, 297, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 311, 312, 313, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 375, 376, 377, 378, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 402, 403, 404, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 420, 421, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 437, 439, 440, 441, 442, 443, 445, 446, 447, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 493, 494, 495, 496, 497, 498, 499, 500, 502, 503, 504, 505, 506, 508, 510, 511, 513, 514, 516, 517, 518, 519, 520, 522, 524, 525, 526, 527, 531, 532, 534, 537, 538, 539, 541, 542, 543, 544, 545, 547, 550, 551, 552, 555, 556, 557, 558, 559, 561, 563, 564, 567, 568, 569, 571, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 613, 614, 615, 616, 617, 618, 620, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 662, 663, 664, 665, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 679, 680, 681, 682, 683, 685, 686, 689, 690, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 710, 711, 712, 713, 714, 715, 716, 717, 719, 721, 722, 723, 724, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 740, 741, 742, 743, 745, 746, 747, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 786, 788, 789, 790, 791, 792, 793, 794, 795, 796, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 850, 851, 852, 853, 854, 855, 856, 857, 858, 860, 861, 862, 863, 864, 865, 867, 869, 870, 871, 872, 873, 874, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 909, 912, 913, 914, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 970, 971, 972, 975, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1061, 1062, 1063, 1064, 1065, 1067, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1085, 1086, 1087, 1088, 1090, 1091, 1092, 1093, 1095, 1096, 1097, 1098, 1099]

In [5]:
# Load datasets
datasets_list = []

if 'Raw' in datasets_name_list:
    RawDs = pd.read_csv(path.join("..","results","extraction","32FINAL.csv"))
    RawDs = RawDs.drop(columns=['Tmp'+str(i) for i in range(100)]) # Exclude Tmp columns
    datasets_list.append(RawDs)
    
if 'Normalized' in datasets_name_list:
    NormDs = pd.read_csv(path.join("..","results","extraction","32FINAL_NORM.csv"))
    NormDs = NormDs.drop(columns=['Tmp'+str(i) for i in range(100)]) # Exclude Tmp columns
    datasets_list.append(NormDs)
    
if 'FFT' in datasets_name_list:  ## NEW
    FFTDs = pd.read_csv(path.join("..","results","extraction","32FINAL_FFT.csv"))
    datasets_list.append(FFTDs)
    
if 'Fourier' in datasets_name_list:
    FourierDs = pd.read_csv(path.join("..","results","extraction","32FINAL_Fourier.csv"))
    datasets_list.append(FourierDs)
    
if 'HOS' in datasets_name_list:
    HOSDs = pd.read_csv(path.join("..","results","extraction","32FINAL_HOS.csv"))
    datasets_list.append(HOSDs)
    
if 'SCM' in datasets_name_list:
    SCMDs = pd.read_csv(path.join("..","results","extraction","32FINAL_NORM_SCM.csv"))
    SCMDs = SCMDs.drop(columns=['AcX'+str(i) for i in range(8)])
#     SCMDs = SCMDs.drop(columns=['AcY'+str(i) for i in range(8)])
    SCMDs = SCMDs.drop(columns=['AcZ'+str(i) for i in range(8)])
    SCMDs = SCMDs.drop(columns=['GyX'+str(i) for i in range(8)])
    SCMDs = SCMDs.drop(columns=['GyY'+str(i) for i in range(8)])
    SCMDs = SCMDs.drop(columns=['GyZ'+str(i) for i in range(8)])
    datasets_list.append(SCMDs)
    
unique, counts = np.unique(SCMDs['Class'], return_counts=True)
print(dict(zip(unique, counts)))
    
# Exclude classes that are not in scope 
for ds in datasets_list:
    if balance_classes:
        ds.drop(list_drops, inplace=True) # Balanced  
    for i in range(6):
        if i not in classes:
            indexNames = ds[ds['Class'] == i].index
            ds.drop(indexNames, inplace=True) # Delete these row indexes from dataframe

# Grouping classes           
if group_classes:
    for ds in datasets_list:
        ds = join_classes(ds, changeClass_dict)
    print("Experiment Classes: ", classes)
    classes = np.unique(list(changeClass_dict.values())).tolist()
    print("New Classes: ", classes)
        
unique, counts = np.unique(SCMDs['Class'], return_counts=True)
print(dict(zip(unique, counts)))

{0: 250, 1: 250, 2: 75, 3: 250, 4: 25, 5: 250}
Experiment Classes:  [0, 2, 3, 4, 5]
New Classes:  [0, 1, 2, 3, 4]
{0: 25, 1: 25, 2: 25, 3: 25, 4: 25}


In [6]:
# if balance_classes:
#     sml_count = np.min(counts)
#     for ds in datasets_list:
#         for c in classes:
#             ds_x = ds[ds['Class'] == c]
#             indexes = list((ds_x).index)
#             index_min = np.min(indexes) + 24
#             index_drop = list(range(index_min+1, np.max(indexes) + 1))
#             if c == 0:
#                 list_index_drop = index_drop
#             else:
#                 for i in index_drop:
#                     list_index_drop.append(i)
#         print(len(list_index_drop))
#         ds.drop(list_index_drop, inplace=True)

#     unique, counts = np.unique(SCMDs['Class'], return_counts=True)
#     print(dict(zip(unique, counts)))

In [7]:
from scipy.stats import randint

# Load classifiers
classifiers_list = [GaussianNB(),        
                    MLPClassifier(max_iter=1000, solver='adam', learning_rate_init=5e-04),      
                    KNeighborsClassifier(),   
                    RandomForestClassifier(),
                    SVC(kernel='linear', probability=True, tol=1e-3),
                    SVC(kernel='poly', probability=True, tol=1e-3),
                    SVC(kernel='rbf', probability=True, tol=1e-3)
                   ]

param_dist_dict = {'Naive_Bayes': [], 
                   'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))},
                   'Nearest_Neighbors': {"n_neighbors": [1,3,5,7,9,11]}, 
                   'Random_Forest': {"n_estimators": [3000],
                                     "max_depth": [6, None],
                                     "max_features": randint(1, 11),
                                     "min_samples_split": randint(2, 11),
                                     "min_samples_leaf": randint(1, 11),
                                     "bootstrap": [True, False],
                                     "criterion": ["gini", "entropy"]},
                    'SVM_Linear': {'kernel': ['linear'], 'C': [2**i for i in range(-5,15)]},
                    'SVM_Polynomial': {'kernel': ['poly'], 'degree': [3, 5, 7 ,9], 'C': [2**i for i in range(-5,15)]},                    
                    'SVM_RBF': {'kernel': ['rbf'], 'gamma': [2**i for i in range(-15,3)],
                                'C': [2**i for i in range(-5,15)]}
                  }

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
import time

current_milli_time = lambda: int(round(time.time() * 1000))

warnings.filterwarnings('ignore')

In [9]:
%%time
if classification_type == 'KFold_Test_Val':
    for n_ds, dataset in enumerate(datasets_list):
        print("processing: " + datasets_name_list[n_ds] + " dataset")
        train_ds_path = path.join(train_path, datasets_name_list[n_ds])
        validation_ds_path = path.join(validation_path, datasets_name_list[n_ds])

        X, y = dataset.iloc[:,:-1], dataset.iloc[:,-1]
        X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)

        list_final_metrics = []
        val_list_final_metrics = []

        for n_clf, classifier in enumerate(classifiers_list):
            classifier_name = classifiers_name_list[n_clf]
            if n_clf != 0:
                random_search = RandomizedSearchCV(classifier, param_dist_dict[classifier_name], cv=4, 
                                                   n_iter=5, scoring='accuracy')
                random_search.fit(X_train, y_train)
                params = random_search.best_params_
                classifier.set_params(**params)

            print("    processing: " + classifier_name + " classifier")
            with open(path.join(train_ds_path, classifier_name+"_config.txt"), 'w') as clf_txt:
                clf_txt.write(str(classifier))

            metrics_dict = dict((k,[]) for k in metrics_name_list)
            val_metrics_dict = dict((k,[]) for k in metrics_name_list)

            cmx = np.zeros((len(classes),len(classes)))

            kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            print("        processing: K-Fold iterations", end='')

            for train_index, test_index in kfold.split(X_train, y_train):
                X_train2, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
                y_train2, y_test = y_train.iloc[train_index], y_train.iloc[test_index]

                fitStart = current_milli_time() # Start fit chronometer
                classifier.fit(X_train2, y_train2)
                fit_time = current_milli_time() - fitStart # Stop fit chronometer and save time

                predictStart = current_milli_time() # Start predict chronometer
                y_predict = classifier.predict(X_test)
                predict_time = current_milli_time() - predictStart # Stop predict chronometer and save time

                accuracy = accuracy_score(y_test, y_predict)
#                 bal_accuracy = balanced_accuracy_score(y_test, y_predict)
                precision = precision_score(y_test, y_predict, average='macro')
                sensitivity = sensitivity_score(y_test, y_predict, average='macro')
                specificity = specificity_score(y_test, y_predict, average='macro')
                f1 = f1_score(y_test, y_predict, average='macro')

                cmx += confusion_matrix(y_test, y_predict)

                metrics_dict["accuracy"].append(accuracy)
#                 metrics_dict["balanced_accuracy"].append(bal_accuracy)
                metrics_dict["precision"].append(precision)
                metrics_dict["sensitivity"].append(sensitivity)
                metrics_dict["specificity"].append(specificity)
                metrics_dict["f1_score"].append(f1)
                metrics_dict["fit_time"].append(fit_time)
                metrics_dict["predict_time"].append(predict_time)

                print('.', end='')

            print(' Done!')
            cmx_csv = pd.DataFrame(cmx.astype(int), index=classes, 
                                   columns=classes).to_csv(path.join(train_ds_path, classifier_name+"_cmx.csv"), sep=',')

            mean_metrics_dict = dict((k+'_mean', np.mean(metrics_dict[k])) for k in metrics_dict.keys())
            std_metrics_dict = dict((k+'_std', np.std(metrics_dict[k])) for k in metrics_dict.keys())

            final_metrics_dict = dict(mean_metrics_dict.items())
            final_metrics_dict.update(std_metrics_dict.items())

            list_final_metrics.append(final_metrics_dict)

            # VALIDATION OF THE MODEL

            val_predictStart = current_milli_time() # Start predict chronometer
            val_y_predict = classifier.predict(X_validation)
            val_predict_time = current_milli_time() - val_predictStart # Stop predict chronometer and save time

            val_accuracy = accuracy_score(y_validation, val_y_predict)
#             val_bal_accuracy = balanced_accuracy_score(y_validation, val_y_predict)
            val_precision = precision_score(y_validation, val_y_predict, average='macro')
            val_sensitivity = sensitivity_score(y_validation, val_y_predict, average='macro')
            val_specificity = specificity_score(y_validation, val_y_predict, average='macro')
            val_f1 = f1_score(y_validation, val_y_predict, average='macro')

            val_cmx = confusion_matrix(y_validation, val_y_predict)

            val_metrics_dict["accuracy"].append(val_accuracy)
#             val_metrics_dict["balanced_accuracy"].append(val_bal_accuracy)
            val_metrics_dict["precision"].append(val_precision)
            val_metrics_dict["sensitivity"].append(val_sensitivity)
            val_metrics_dict["specificity"].append(val_specificity)
            val_metrics_dict["f1_score"].append(val_f1)
            val_metrics_dict["fit_time"].append(np.mean(metrics_dict["fit_time"]))
            val_metrics_dict["predict_time"].append(val_predict_time)

            val_cmx_csv = pd.DataFrame(val_cmx.astype(int), index=classes, 
                                       columns=classes).to_csv(path.join(validation_ds_path, classifier_name+"_cmx.csv"), sep=',')

            val_mean_metrics_dict = dict((k, np.mean(val_metrics_dict[k])) for k in val_metrics_dict.keys())

            val_list_final_metrics.append(val_mean_metrics_dict)

        metrics_csv = pd.DataFrame(list_final_metrics, 
                                   index=classifiers_name_list).to_csv(path.join(train_ds_path, "evaluation_metrics.csv"), sep=',')

        val_metrics_csv = pd.DataFrame(val_list_final_metrics, 
                                       index=classifiers_name_list).to_csv(path.join(validation_ds_path, "evaluation_metrics.csv"), sep=',')

    print('\n\nFinished!')

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 14.8 µs


In [10]:
%%time
if classification_type == 'KFold_Test':
    for n_ds, dataset in enumerate(datasets_list):
        print("processing: " + datasets_name_list[n_ds] + " dataset")
        train_ds_path = path.join(train_path, datasets_name_list[n_ds])

        X, y = dataset.iloc[:,:-1], dataset.iloc[:,-1]

        list_final_metrics = []

        for n_clf, classifier in enumerate(classifiers_list):
            classifier_name = classifiers_name_list[n_clf]
            if n_clf != 0 and n_clf != 5 and n_clf != 6:
                random_search = RandomizedSearchCV(classifier, param_dist_dict[classifier_name], cv=4, 
                                                   n_iter=5, scoring='accuracy')
                random_search.fit(X, y)
                params = random_search.best_params_
                classifier.set_params(**params)

            print("    processing: " + classifier_name + " classifier")
            with open(path.join(train_ds_path, classifier_name+"_config.txt"), 'w') as clf_txt:
                clf_txt.write(str(classifier))

            metrics_dict = dict((k,[]) for k in metrics_name_list)

            cmx = np.zeros((len(classes),len(classes)))

            kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            print("        processing: K-Fold iterations", end='')

            for train_index, test_index in kfold.split(X, y):
                if n_clf != 0 and n_clf != 5 and n_clf != 5:
                    classifier_aux = classifier.set_params(**params) ############
                else: 
                    classifier_aux = classifier

                X_train, X_test = X.iloc[train_index], X.iloc[test_index] ####
                y_train, y_test = y.iloc[train_index], y.iloc[test_index] ####
                
                if n_clf >= 5: 
                    X_train = (X_train - X_train.mean()) / X_train.std()
                    X_test = (X_test - X_test.mean()) / X_test.std()

                fitStart = current_milli_time() # Start fit chronometer
                classifier_aux.fit(X_train, y_train) #####
                fit_time = current_milli_time() - fitStart # Stop fit chronometer and save time

                predictStart = current_milli_time() # Start predict chronometer
                y_predict = classifier_aux.predict(X_test) #####
                predict_time = current_milli_time() - predictStart # Stop predict chronometer and save time
                
                accuracy = accuracy_score(y_test, y_predict)
#                 bal_accuracy = balanced_accuracy_score(y_test, y_predict)                
                precision = precision_score(y_test, y_predict, average='macro')                
                sensitivity = sensitivity_score(y_test, y_predict, average='macro')                
                specificity = specificity_score(y_test, y_predict, average='macro')                
                f1 = f1_score(y_test, y_predict, average='macro')

                cmx += confusion_matrix(y_test, y_predict)

                metrics_dict["accuracy"].append(accuracy)
#                 metrics_dict["balanced_accuracy"].append(bal_accuracy)
                metrics_dict["precision"].append(precision)
                metrics_dict["sensitivity"].append(sensitivity)
                metrics_dict["specificity"].append(specificity)
                metrics_dict["f1_score"].append(f1)
                metrics_dict["fit_time"].append(fit_time)
                metrics_dict["predict_time"].append(predict_time)

                print('.', end='')

            print(' Done!')
            cmx_csv = pd.DataFrame(cmx.astype(int), index=classes, 
                                   columns=classes).to_csv(path.join(train_ds_path, classifier_name+"_cmx.csv"), sep=',')

            mean_metrics_dict = dict((k+'_mean', np.mean(metrics_dict[k])) for k in metrics_dict.keys())
            std_metrics_dict = dict((k+'_std', np.std(metrics_dict[k])) for k in metrics_dict.keys())

            final_metrics_dict = dict(mean_metrics_dict.items()) 

            final_metrics_dict.update(std_metrics_dict.items())

            list_final_metrics.append(final_metrics_dict)

        metrics_csv = pd.DataFrame(list_final_metrics, 
                                   index=classifiers_name_list).to_csv(path.join(train_ds_path, "evaluation_metrics.csv"), sep=',')

    print('\n\nFinished!')

processing: SCM dataset
    processing: Naive_Bayes classifier
        processing: K-Fold iterations.......... Done!
    processing: MLP classifier
        processing: K-Fold iterations.......... Done!
    processing: Nearest_Neighbors classifier
        processing: K-Fold iterations.......... Done!
    processing: Random_Forest classifier
        processing: K-Fold iterations.......... Done!
    processing: SVM_Linear classifier
        processing: K-Fold iterations.......... Done!
    processing: SVM_Polynomial classifier
        processing: K-Fold iterations.......... Done!
    processing: SVM_RBF classifier
        processing: K-Fold iterations.......... Done!


Finished!
CPU times: user 5min 2s, sys: 1min 43s, total: 6min 45s
Wall time: 3min 19s


In [11]:
%%time
if classification_type == "HoldOut":
    for n_ds, dataset in enumerate(datasets_list):
        print("processing: " + datasets_name_list[n_ds] + " dataset")
        train_ds_path = path.join(train_path, datasets_name_list[n_ds])

        X, y = dataset.iloc[:,:-1], dataset.iloc[:,-1]
        list_final_metrics = []

        for n_clf, classifier in enumerate(classifiers_list):
            classifier_name = classifiers_name_list[n_clf]
            if n_clf != 0:
                random_search = RandomizedSearchCV(classifier, param_dist_dict[classifier_name], cv=4, 
                                                   n_iter=5, scoring='accuracy')
                random_search.fit(X, y)
                params = random_search.best_params_
                classifier.set_params(**params)

            print("    processing: " + classifier_name + " classifier")
            with open(path.join(train_ds_path, classifier_name+"_config.txt"), 'w') as clf_txt:
                clf_txt.write(str(classifier))

            metrics_dict = dict((k,[]) for k in metrics_name_list)

            cmx = np.zeros((len(classes),len(classes)))
            
            print("        processing: Hold-Out iterations", end='')
            
            for rnd in range(100):
                if n_clf != 0:
                    classifier_aux = classifier.set_params(**params)
                else: 
                    classifier_aux = classifier
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)

                fitStart = current_milli_time() # Start fit chronometer
                classifier_aux.fit(X_train, y_train)
                fit_time = current_milli_time() - fitStart # Stop fit chronometer and save time

                predictStart = current_milli_time() # Start predict chronometer
                y_predict = classifier_aux.predict(X_test)
                predict_time = current_milli_time() - predictStart # Stop predict chronometer and save time

                accuracy = accuracy_score(y_test, y_predict)
#                 bal_accuracy = balanced_accuracy_score(y_test, y_predict)
                precision = precision_score(y_test, y_predict, average='macro')
                sensitivity = sensitivity_score(y_test, y_predict, average='macro')
                specificity = specificity_score(y_test, y_predict, average='macro')
                f1 = f1_score(y_test, y_predict, average='macro')

                cmx += confusion_matrix(y_test, y_predict)

                metrics_dict["accuracy"].append(accuracy)
#                 metrics_dict["balanced_accuracy"].append(bal_accuracy)
                metrics_dict["precision"].append(precision)
                metrics_dict["sensitivity"].append(sensitivity)
                metrics_dict["specificity"].append(specificity)
                metrics_dict["f1_score"].append(f1)
                metrics_dict["fit_time"].append(fit_time)
                metrics_dict["predict_time"].append(predict_time)

                print('.', end='')

            print(' Done!')
            cmx_csv = pd.DataFrame(cmx.astype(int), index=classes, 
                                   columns=classes).to_csv(path.join(train_ds_path, classifier_name+"_cmx.csv"), sep=',')

            mean_metrics_dict = dict((k+'_mean', np.mean(metrics_dict[k])) for k in metrics_dict.keys())
            std_metrics_dict = dict((k+'_std', np.std(metrics_dict[k])) for k in metrics_dict.keys())

            final_metrics_dict = dict(mean_metrics_dict.items())
            final_metrics_dict.update(std_metrics_dict.items())

            list_final_metrics.append(final_metrics_dict)

        metrics_csv = pd.DataFrame(list_final_metrics, 
                                   index=classifiers_name_list).to_csv(path.join(train_ds_path, "evaluation_metrics.csv"), sep=',')

    print('\n\nFinished!')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs
